In [76]:
# Project 2

# Implementing: nearest neighbor
# Calculate euclidiean dist
import math
import numpy as np
import copy
# Use Pandas for reading txt files
import pandas as pd

# For nearest neighbor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import GPUtil 
GPUtil.getAvailable()
import torch 
use_cuda = torch.cuda. is_available()
from numba import jit, cuda,numba
import logging;
logging.disable(logging.WARNING)
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

 
     
# Load data
data = np.loadtxt('/home/foocypher/cs170/CS_170-Project-2/CS170_Large_Data__21.txt')

####################
# Cross-Validation #
####################
@jit
def nearestneighbor(data, current_set_of_features, j,algo):
    test_features=list(current_set_of_features)
    if(algo==1):
        test_features.append(j)
    if(algo==2):##elimination or whatever
        test_features.remove(j)
    number_correctly_classfied = 0
    
    result=0 # of what?
    for i in data:
        # Object to classify
        short_man=math.inf#local shortest distance for that variable or somfin

        for k in data:
          if not np.array_equal(k, i):
                distance =0
                for l in test_features:
                    distance += pow((i[l] - k[l]), 2.0)
                 
                if math.sqrt(distance) < short_man:
                    short_man = math.sqrt(distance)
                    
                    result =k[0]

        if result == i[0]:
            number_correctly_classfied = 1 +number_correctly_classfied

    accuracy = number_correctly_classfied / data.shape[0] #or (len(data) - 1)
    
    return accuracy

#####################
# Forward Selection #
#####################
def forwardSelection(data):
  current_set_of_features = []   # Current feature list is empty
  best_accuracy = 0
  current_best_features = []    # Highest accuracy set of features

  for i in range(1,len(data[0])):  # Traverse through number of features
    
    print ("On Level " + str(i) + " of the search tree:")
    feature_to_add_on_this_level = -1 # Feature with best accuracy in the level
    best_accuracy_so_far = 0 
             # Level accuracy
    for j in range(1,len(data[i])):
      if j not in current_set_of_features:
        
        accuracy = nearestneighbor(data,current_set_of_features,j,1)
        if not current_set_of_features:
          print ("---Using feature(s) {" + str(j) + "} accuracy is " + str(accuracy) + "%")
        else:
          print( "---Using feature(s) {" + str(j) + ", " + "".join(str(current_set_of_features)) + "} accuracy is " + str(accuracy) + "%")

        if(accuracy > best_accuracy_so_far):
          best_accuracy_so_far = accuracy
          feature_to_add_on_this_level = j

    if(best_accuracy_so_far > best_accuracy):
      best_accuracy = best_accuracy_so_far
      current_best_features.append(feature_to_add_on_this_level)
    else:
      print ("\n(Warning, Accuracy has decreased! Continuing search in case of local maxima)")

    if(feature_to_add_on_this_level != -1):
      current_set_of_features.append(feature_to_add_on_this_level)
      print ("Feature set " + "".join(str(current_set_of_features)) + " was best with an accuracy of " + str(best_accuracy_so_far) + "%\n")

  print ("Best feature(s): " + "".join(str(current_best_features)) + ", with an accuracy of " + str(best_accuracy) + "%")

########################
# Backward Elimination #
########################
@jit
def backward_elimination(data):
  # Initialize current set of features as all features
  current_set_of_features = [i for i in range(1, len(data[0]))]
  current_best_features = []
  # Initialize best accuracy to 0
  best_so_far_accuracy = 0

  for i in range(1, data.shape[0]-1):
    feature_to_remove_at_this_level = 0
    best_accuracy = 0
    # Create a new set of features by removing k
    # new_set_of_features = current_set_of_features[:]
    # new_set_of_features.remove(k)

    for k in range(1, data.shape[0]-1):
      # Iterate over features
      if k in current_set_of_features:
        print(f'---Using feature(s) {k}')
        accuracy = nearestneighbor(data, current_set_of_features, k, 2)

        # Update best accuracy and feature to remove
        if accuracy > best_accuracy:
          best_accuracy = accuracy
          feature_to_remove_at_this_level = k

    # Check if removing the feature improves accuracy
    if feature_to_remove_at_this_level in current_set_of_features:
      # Update current set of features
      current_set_of_features.remove(feature_to_remove_at_this_level)
      print( "---Feature(s) {" + str(k) + ", " + "".join(str(current_set_of_features)) + "} removed.")
      print ("Feature set " + "".join(str(current_set_of_features)) + " was best with an accuracy of " + str(best_so_far_accuracy) + "%\n")

    if best_accuracy >= best_so_far_accuracy:
      best_so_far_accuracy = best_accuracy
      current_best_features = list(current_set_of_features)

  # Return final set of features and best accuracy
  print ("Best feature(s): " + "".join(str(current_best_features)) + ", with an accuracy of " + str(best_so_far_accuracy) + "%")
  return
      
##########
# Driver #
##########
def main():

  # UI
  c=int(input("Which algorithm? 1 for Forward Selection, 2 for Backward Elimination \n"))

  if c==1:
    # Perform Forward Selection
    accuracy = forwardSelection(data)
    
    
  elif c==2:
    # Perform Backward Elimination
    backward_elimination(data)
    
    
main()

/tmp/ipykernel_6324/2118313366.py:112: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "backward_elimination" failed type inference due to: No implementation of function Function(<class 'str'>) found for signature:
 
 >>> str(list(int64)<iv=None>)
 
There are 10 candidate implementations:
      - Of which 10 did not match due to:
      Overload of function 'str': File: <numerous>: Line N/A.
        With argument(s): '(list(int64)<iv=None>)':
       No match.

During: resolving callee type: Function(<class 'str'>)
During: typing of call at /tmp/ipykernel_6324/2118313366.py (142)


File "../../../../tmp/ipykernel_6324/2118313366.py", line 142:
<source missing, REPL/exec in use?>

  @jit
/tmp/ipykernel_6324/2118313366.py:112: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "backward_elimination" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatche

---Using feature(s) 1
---Using feature(s) 2
---Using feature(s) 3
---Using feature(s) 4
---Using feature(s) 5
---Using feature(s) 6
---Using feature(s) 7
---Using feature(s) 8
---Using feature(s) 9
---Using feature(s) 10
---Using feature(s) 11
---Using feature(s) 12
---Using feature(s) 13
---Using feature(s) 14
---Using feature(s) 15
---Using feature(s) 16
---Using feature(s) 17
---Using feature(s) 18
---Using feature(s) 19
---Using feature(s) 20
---Using feature(s) 21
---Using feature(s) 22
---Using feature(s) 23
---Using feature(s) 24
---Using feature(s) 25
---Using feature(s) 26
---Using feature(s) 27
---Using feature(s) 28
---Using feature(s) 29
---Using feature(s) 30
---Using feature(s) 31
---Using feature(s) 32
---Using feature(s) 33
---Using feature(s) 34
---Using feature(s) 35
---Using feature(s) 36
---Using feature(s) 37
---Using feature(s) 38
---Using feature(s) 39
---Using feature(s) 40
---Feature(s) {998, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2